In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from sklearn.utils.class_weight import compute_class_weight
import numpy as np


In [3]:
# Set up device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Data transformations for image processing
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
# Custom Dataset Class for Deepfake Images
class DeepfakeImageDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.classes = {'Real': 0, 'Fake': 1}
        self.data = []

        for label in self.classes:
            class_dir = os.path.join(root_dir, label)
            for img_name in os.listdir(class_dir):
                img_path = os.path.join(class_dir, img_name)
                self.data.append((img_path, self.classes[label]))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path, label = self.data[idx]
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, label


In [4]:
# Define dataset directories (adjust these paths)
train_dir = '/content/drive/MyDrive/Deep_learning/TrimDataset/Dataset/Train'
val_dir = '/content/drive/MyDrive/Deep_learning/TrimDataset/Dataset/Validation'
test_dir = '/content/drive/MyDrive/Deep_learning/TrimDataset/Dataset/Test'

# Load datasets
train_dataset = DeepfakeImageDataset(train_dir, transform=transform)
val_dataset = DeepfakeImageDataset(val_dir, transform=transform)
test_dataset = DeepfakeImageDataset(test_dir, transform=transform)

# DataLoader for batch processing
test_loader = DataLoader(train_dataset, batch_size=32, shuffle=False)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
train_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)


In [5]:
# ResNet Feature Extractor
class ResNetFeatureExtractor(nn.Module):
    def __init__(self):
        super(ResNetFeatureExtractor, self).__init__()
        self.resnet = models.resnet50(pretrained=True)
        self.resnet.fc = nn.Identity()  # Remove final classification layer

    def forward(self, x):
        return self.resnet(x)


# LSTM Classifier Model
class LSTMClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes, dropout=0.5):
        super(LSTMClassifier, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        h0 = torch.zeros(2, x.size(0), 128).to(device)  # Initial hidden state
        c0 = torch.zeros(2, x.size(0), 128).to(device)  # Initial cell state
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])  # Output from last LSTM step
        return out



In [6]:
# Initialize models
feature_extractor = ResNetFeatureExtractor().to(device)
lstm_model = LSTMClassifier(input_size=2048, hidden_size=128, num_layers=2, num_classes=2).to(device)

# Class weights based on real/fake distribution
y_train = [label for _, label in train_dataset]
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
class_weights_tensor = torch.tensor(class_weights, dtype=torch.float).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss(weight=class_weights_tensor)  # Class weights added
optimizer = torch.optim.Adam(lstm_model.parameters(), lr=1e-4)


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 146MB/s]


In [ ]:
# Training settings
epochs = 10
patience = 3  # Early stopping patience
best_test_acc = 0  # Track best test accuracy for model saving
trigger_times = 0

# Custom threshold for classification
threshold = 0.5

for epoch in range(epochs):
    # Training phase
    lstm_model.train()
    total_loss, total_correct = 0, 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        # Extract features with ResNet
        with torch.no_grad():  # Freeze ResNet weights
            features = feature_extractor(images).unsqueeze(1)  # (batch, seq_len, features)

        # LSTM classification
        outputs = lstm_model(features)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        _, predicted = outputs.max(1)
        total_correct += predicted.eq(labels).sum().item()

    train_accuracy = total_correct / len(train_dataset)
    train_loss = total_loss / len(train_loader)

    # Testing phase at the end of each epoch
    lstm_model.eval()
    test_correct = 0
    test_total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)

            # Extract features with ResNet
            features = feature_extractor(images).unsqueeze(1)

            # LSTM model prediction
            outputs = lstm_model(features)

            # Apply threshold to classify as real or fake
            probs = torch.softmax(outputs, dim=1)
            predicted = (probs[:, 1] >= threshold).long()

            # Count correct predictions
            test_correct += predicted.eq(labels).sum().item()
            test_total += labels.size(0)

    test_accuracy = test_correct / test_total



    print(f"Epoch [{epoch+1}/{epochs}], Train Loss: {train_loss:.4f}, Train Acc: {train_accuracy:.4f}, "
          f"Test Acc: {test_accuracy:.4f}")

    # Early stopping check based on test accuracy
    if test_accuracy > best_test_acc:
        best_test_acc = test_accuracy
        trigger_times = 0
        torch.save(lstm_model.state_dict(), '/content/drive/MyDrive/DL/best_lstm_model.pth')  # Save the best model
    else:
        trigger_times += 1
        if trigger_times >= patience:
            print("Early stopping triggered.")
            break


Epoch [1/10], Train Loss: 0.3083, Train Acc: 0.8749, Test Acc: 0.5141


In [ ]:
# Save the trained LSTM model's state dictionary
torch.save(lstm_model.state_dict(), "/content/drive/MyDrive/DL/FinalLSTM_model.pth")


In [ ]:
# Load the trained model if you saved it
lstm_model.load_state_dict(torch.load('/content/drive/MyDrive/DL/FinalLSTM_model.pth'))
lstm_model.eval()  # Set model to evaluation mode


<ipython-input-12-476c61195496>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  lstm_model.load_state_dict(torch.load('/content/drive/MyDrive/DL/FinalLSTM_model.pth'))


LSTMClassifier(
  (lstm): LSTM(2048, 128, num_layers=2, batch_first=True, dropout=0.5)
  (fc): Linear(in_features=128, out_features=2, bias=True)
)

In [ ]:
# Transformation pipeline for inference
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


In [ ]:
# Load the trained models
feature_extractor = ResNetFeatureExtractor().to(device)
feature_extractor.eval()  # Set to evaluation mode

lstm_model = LSTMClassifier(input_size=2048, hidden_size=128, num_layers=2, num_classes=2).to(device)
lstm_model.load_state_dict(torch.load("/content/drive/MyDrive/DL/FinalLSTM_model.pth"))
lstm_model.eval()  # Set to evaluation mode

# Load and preprocess the image
image_path = '/content/drive/MyDrive/Deep_learning/TrimDataset/Dataset/Validation/Fake/fake_2070.jpg'
image = Image.open(image_path).convert('RGB')
image = transform(image).unsqueeze(0).to(device)  # Add batch dimension and send to device

# Step 1: Extract features using ResNet
with torch.no_grad():
    features = feature_extractor(image).unsqueeze(1)  # (batch, seq_len, features)

# Step 2: Classify using the LSTM
with torch.no_grad():
    output = lstm_model(features)
    _, predicted = torch.max(output, 1)

# Interpret the result
classes = {0: 'Real', 1: 'Fake'}
print(f"Prediction: {classes[predicted.item()]}")

<ipython-input-22-b9ec7fc0d458>:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  lstm_model.load_state_dict(torch.load("/content/drive/MyDrive/DL/FinalLSTM_model.pth"))


Prediction: Real
